In [ ]:
%pip install -U transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 575.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 40.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2


In [ ]:

%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [ ]:

import wandb

wb_token = 'ddcc40dc59b9e6279bcdac9f053a6f9addbf80db'
wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune llama-3.1-8b-it on Sentiment Analysis Dataset',
    job_type="training",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 3malwat (3malwat-iit-gandhinagar). Use `wandb login --relogin` to force relogin


In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

from datasets import load_dataset

In [ ]:
dataset = load_dataset("sst2", split="train")
# load the dataset into a dataframe
df = pd.DataFrame(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
df = df.drop(columns=['idx'])
df

,sentence,label
0,hide new secretions from the parental units,0
1,"contains no wit , only labored gags",0
2,that loves its characters and communicates som...,1
3,remains utterly satisfied to remain the same t...,0
4,on the worst revenge-of-the-nerds clichés the ...,0
...,...,...
67344,a delightful comedy,1
67345,"anguish , anger and frustration",0
67346,"at achieving the modest , crowd-pleasing goals...",1
67347,a patient viewer,1


In [ ]:
# Shuffle the DataFrame and select only 3000 rows
df = df.sample(frac=1, random_state=1).reset_index(drop=True)

# Split the DataFrame
train_size = 0.7
eval_size = 0.1

# Calculate sizes
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))


# Split the data
X_train = df[:train_end]
X_eval = df[train_end:eval_end]
X_test = df[eval_end:]

# Define the prompt generation functions for SST-2 dataset
def generate_prompt(data_point):
    return f"""
            Classify the sentiment of the text as Positive (1) or Negative (0) and return the corresponding sentiment label.
text: {data_point["sentence"]}
label: {data_point["label"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
            Classify the sentiment of the text as Positive (1) or Negative (0) and return the corresponding sentiment label.
text: {data_point["sentence"]}
label: """.strip()

# Generate prompts for training and evaluation data
X_train.loc[:,'sentence'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)

# Generate test prompts and extract true labels
y_true = X_test.loc[:,'label']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["sentence"])

<ipython-input-7-e67f0c2265d4>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)


In [ ]:
X_train.label.value_counts()

,count
label,
1,26336
0,20808


In [ ]:
# Convert to datasets
train_data = Dataset.from_pandas(X_train[["sentence"]])
eval_data = Dataset.from_pandas(X_eval[["sentence"]])

In [ ]:
train_data['sentence'][3]

'Classify the sentiment of the text as Positive (1) or Negative (0) and return the corresponding sentiment label.\ntext: it may not be particularly innovative , \nlabel: 0'

In [ ]:
from huggingface_hub import login
login()

In [ ]:
base_model_name = "meta-llama/Llama-3.2-1B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

# LLAMA 3.2 1b parameter calculation

In [ ]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        # if not parameter.requires_grad:
        #     continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model)

+--------------------------------------------------------+------------+
|                        Modules                         | Parameters |
+--------------------------------------------------------+------------+
|               model.embed_tokens.weight                | 262668288  |
|   model.layers.0.self_attn.q_proj.base_layer.weight    |  2097152   |
| model.layers.0.self_attn.q_proj.lora_A.default.weight  |   131072   |
| model.layers.0.self_attn.q_proj.lora_B.default.weight  |   131072   |
|   model.layers.0.self_attn.k_proj.base_layer.weight    |   524288   |
| model.layers.0.self_attn.k_proj.lora_A.default.weight  |   131072   |
| model.layers.0.self_attn.k_proj.lora_B.default.weight  |   32768    |
|   model.layers.0.self_attn.v_proj.base_layer.weight    |   524288   |
| model.layers.0.self_attn.v_proj.lora_A.default.weight  |   131072   |
| model.layers.0.self_attn.v_proj.lora_B.default.weight  |   32768    |
|   model.layers.0.self_attn.o_proj.base_layer.weight    |  2097

794363904

In [ ]:
def predict(test, model, tokenizer):
    y_pred = []
    categories = ["0", "1"]

    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["sentence"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=2,
                        temperature=0.1)

        result = pipe(prompt)
        answer = result[0]['generated_text'].split("label:")[-1].strip()

        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")

    return y_pred



In [ ]:
def evaluate(y_true, y_pred):
    labels = ["0","1"]
    mapping = {label: idx for idx, label in enumerate(labels)}

    def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found, but should not occur with correct data

    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true_mapped)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {label_accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true_mapped, y_pred=y_pred_mapped, target_names=labels, labels=list(range(len(labels))))
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true_mapped, y_pred=y_pred_mapped, labels=list(range(len(labels))))
    print('\nConfusion Matrix:')
    print(conf_matrix)



# Model evaluation before fine-tuning


In [ ]:
y_pred = predict(X_test, model, tokenizer)

100%|██████████| 13471/13471 [19:11<00:00, 11.70it/s]


In [ ]:
y_true_lst = [str(i) for i in y_true]

In [ ]:
evaluate(y_true_lst, y_pred)

Accuracy: 0.360
Accuracy for label 0: 0.461
Accuracy for label 1: 0.279

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.46      0.58      5953
           1       0.80      0.28      0.41      7518

   micro avg       0.78      0.36      0.49     13471
   macro avg       0.78      0.37      0.49     13471
weighted avg       0.78      0.36      0.49     13471


Confusion Matrix:
[[2747  529]
 [ 840 2100]]


In [ ]:
# save the csv file with the prompt, true label and predicted label without finetuning

# Check if the lengths of all columns are equal
if len(X_test["sentence"]) == len(y_true) == len(y_pred):
    print("Lengths are equal. Proceeding to save results.")

    # Create a DataFrame for the results
    results_df = pd.DataFrame({
        "Prompt": X_test["sentence"],  # The generated prompts
        "True Label": y_true,         # The true labels from the test set
        "Predicted Label": y_pred     # The predictions from the model
    })

    # Save the DataFrame to a CSV file
    results_file = "sst2_predictions_without_finetuning.csv"
    results_df.to_csv(results_file, index=False)

    print(f"Results saved to {results_file}")
else:
    print("Error: The lengths of the columns are not equal!")
    print(f"Prompts length: {len(X_test['sentence'])}")
    print(f"True labels length: {len(y_true)}")
    print(f"Predicted labels length: {len(y_pred)}")


Lengths are equal. Proceeding to save results.
Results saved to sst2_predictions_without_finetuning.csv


In [ ]:
# prompt: find value counts of predicted labels

from collections import Counter

# Assuming y_pred is a list of predicted labels
label_counts = Counter(y_pred)

label_counts

Counter({'none': 7255, '1': 2629, '0': 3587})

# Finetuning the model


In [ ]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
modules = find_all_linear_names(model)
modules

['gate_proj', 'o_proj', 'up_proj', 'k_proj', 'down_proj', 'q_proj', 'v_proj']

In [ ]:
output_dir="llama-3.2-fine-tuned-model"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=1,                       # number of training epochs
    per_device_train_batch_size=16,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    # report_to="wandb",                  # report metrics to w&b
    eval_strategy="steps",              # save checkpoint every epoch
    eval_steps = 0.2
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="sentence",
    tokenizer=tokenizer,
    max_seq_length=512,
    packing=False,
    dataset_kwargs={
    "add_special_tokens": False,
    "append_concat_token": False,
    }
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_kwargs. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:334: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you pas

Map:   0%|          | 0/47144 [00:00<?, ? examples/s]

Map:   0%|          | 0/6734 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
74,1.124000,7.474593
148,1.030400,7.360301
222,1.032600,7.222432
296,1.006800,7.183551


TrainOutput(global_step=368, training_loss=1.111211513209602, metrics={'train_runtime': 3009.2445, 'train_samples_per_second': 15.666, 'train_steps_per_second': 0.122, 'total_flos': 1.822128873996288e+16, 'train_loss': 1.111211513209602, 'epoch': 0.9989820156090939})

In [ ]:
wandb.finish()
model.config.use_cache = True

eval/loss,█▅▂▁
eval/runtime,█▁▁▁
eval/samples_per_second,▁███
eval/steps_per_second,▁███
train/epoch,▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/grad_norm,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▆█████▇▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
train/loss,██▇▃▃▂▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,7.18355
eval/runtime,114.1883


In [ ]:
# Save trained model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('llama-3.2-fine-tuned-model/tokenizer_config.json',
 'llama-3.2-fine-tuned-model/special_tokens_map.json',
 'llama-3.2-fine-tuned-model/tokenizer.json')

In [ ]:
# zip a folder
!zip -r llama-3.2-fine-tuned-model.zip /content/llama-3.2-fine-tuned-model

  adding: content/llama-3.2-fine-tuned-model/ (stored 0%)
  adding: content/llama-3.2-fine-tuned-model/runs/ (stored 0%)
  adding: content/llama-3.2-fine-tuned-model/runs/Nov20_13-52-33_428ff2a4c514/ (stored 0%)
  adding: content/llama-3.2-fine-tuned-model/runs/Nov20_13-52-33_428ff2a4c514/events.out.tfevents.1732110763.428ff2a4c514.627.0 (deflated 69%)
  adding: content/llama-3.2-fine-tuned-model/README.md (deflated 66%)
  adding: content/llama-3.2-fine-tuned-model/checkpoint-368/ (stored 0%)
  adding: content/llama-3.2-fine-tuned-model/checkpoint-368/README.md (deflated 66%)
  adding: content/llama-3.2-fine-tuned-model/checkpoint-368/adapter_config.json (deflated 54%)
  adding: content/llama-3.2-fine-tuned-model/checkpoint-368/tokenizer.json (deflated 85%)
  adding: content/llama-3.2-fine-tuned-model/checkpoint-368/adapter_model.safetensors (deflated 8%)
  adding: content/llama-3.2-fine-tuned-model/checkpoint-368/special_tokens_map.json (deflated 64%)
  adding: content/llama-3.2-fine-

In [ ]:
# prompt: connect to drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/llama-3.2-fine-tuned-model.zip /content/drive/MyDrive/assign_6_finetuned

# LLAMA 3.2 1b parameter calculation

In [ ]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        # if not parameter.requires_grad:
        #     continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model)

+--------------------------------------------------------+------------+
|                        Modules                         | Parameters |
+--------------------------------------------------------+------------+
|               model.embed_tokens.weight                | 262668288  |
|   model.layers.0.self_attn.q_proj.base_layer.weight    |  2097152   |
| model.layers.0.self_attn.q_proj.lora_A.default.weight  |   131072   |
| model.layers.0.self_attn.q_proj.lora_B.default.weight  |   131072   |
|   model.layers.0.self_attn.k_proj.base_layer.weight    |   524288   |
| model.layers.0.self_attn.k_proj.lora_A.default.weight  |   131072   |
| model.layers.0.self_attn.k_proj.lora_B.default.weight  |   32768    |
|   model.layers.0.self_attn.v_proj.base_layer.weight    |   524288   |
| model.layers.0.self_attn.v_proj.lora_A.default.weight  |   131072   |
| model.layers.0.self_attn.v_proj.lora_B.default.weight  |   32768    |
|   model.layers.0.self_attn.o_proj.base_layer.weight    |  2097

794363904

# Evaluating the finetuned model

In [ ]:
y_pred = predict(X_test, model, tokenizer)


  0%|          | 0/13471 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 13471/13471 [45:44<00:00,  4.91it/s]


In [ ]:
y_true_lst = [str(i) for i in y_true]


In [ ]:
evaluate(y_true_lst, y_pred)

Accuracy: 0.944
Accuracy for label 0: 0.953
Accuracy for label 1: 0.937

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.95      0.94      5953
           1       0.96      0.94      0.95      7518

    accuracy                           0.94     13471
   macro avg       0.94      0.95      0.94     13471
weighted avg       0.94      0.94      0.94     13471


Confusion Matrix:
[[5673  280]
 [ 472 7046]]


In [ ]:
# save the csv file with the prompt, true label and predicted label without finetuning

# Check if the lengths of all columns are equal
if len(X_test["sentence"]) == len(y_true) == len(y_pred):
    print("Lengths are equal. Proceeding to save results.")

    # Create a DataFrame for the results
    results_df = pd.DataFrame({
        "Prompt": X_test["sentence"],  # The generated prompts
        "True Label": y_true,         # The true labels from the test set
        "Predicted Label": y_pred     # The predictions from the model
    })

    # Save the DataFrame to a CSV file
    results_file = "sst2_predictions_with_finetuning.csv"
    results_df.to_csv(results_file, index=False)

    print(f"Results saved to {results_file}")
else:
    print("Error: The lengths of the columns are not equal!")
    print(f"Prompts length: {len(X_test['sentence'])}")
    print(f"True labels length: {len(y_true)}")
    print(f"Predicted labels length: {len(y_pred)}")


Lengths are equal. Proceeding to save results.
Results saved to sst2_predictions_with_finetuning.csv


In [ ]:
!cp -r llama-3.2-fine-tuned-model.zip /content/llama3_2finetuned-model.zip

# **Final Evaluation**

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_classification_results(csv_file):
    """
    Evaluate classification results based on the saved CSV file.
    Args:
        csv_file (str): Path to the CSV file containing "True Label" and "Predicted Label" columns.
    Returns:
        dict: A dictionary containing evaluation metrics.
    """
    # Load the CSV file
    try:
        results_df = pd.read_csv(csv_file)
    except FileNotFoundError:
        print(f"Error: File '{csv_file}' not found.")
        return

    # Check if necessary columns exist
    if not {"True Label", "Predicted Label"}.issubset(results_df.columns):
        print("Error: CSV file must contain 'True Label' and 'Predicted Label' columns.")
        return

    # Extract true and predicted labels and convert to strings
    y_true = results_df["True Label"].astype(str)
    y_pred = results_df["Predicted Label"].astype(str)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    # Get total rows
    total_rows = len(results_df)

    # Count occurrences of each class
    true_label_counts = y_true.value_counts()
    predicted_label_counts = y_pred.value_counts()
    # Compile results
    evaluation_results = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    }

    # Print results
    print("Evaluation Results:")
    print(f"Total Rows: {total_rows}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (Weighted): {precision:.4f}")
    print(f"Recall (Weighted): {recall:.4f}")
    print(f"F1 Score (Weighted): {f1:.4f}")
    print("\nTrue Label Counts:")
    print(true_label_counts)
    print("\nPredicted Label Counts:")
    print(predicted_label_counts)
    for metric, value in evaluation_results.items():
        print(f"{metric}: {value:.4f}")

    return evaluation_results


## Before finetuning results

In [ ]:
csv_file = "sst2_predictions_without_finetuning.csv"
zero_shot_evaluation_results = evaluate_classification_results(csv_file)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation Results:
Total Rows: 13471
Accuracy: 0.3598
Precision (Weighted): 0.7842
Recall (Weighted): 0.3598
F1 Score (Weighted): 0.4855

True Label Counts:
True Label
1    7518
0    5953
Name: count, dtype: int64

Predicted Label Counts:
Predicted Label
none    7255
0       3587
1       2629
Name: count, dtype: int64
Accuracy: 0.3598
Precision: 0.7842
Recall: 0.3598
F1 Score: 0.4855


## After Finetuning:

In [ ]:
csv_file = "sst2_predictions_with_finetuning.csv"
finetuned_evaluation_results = evaluate_classification_results(csv_file)


Evaluation Results:
Total Rows: 13471
Accuracy: 0.9442
Precision (Weighted): 0.9447
Recall (Weighted): 0.9442
F1 Score (Weighted): 0.9443

True Label Counts:
True Label
1    7518
0    5953
Name: count, dtype: int64

Predicted Label Counts:
Predicted Label
1    7326
0    6145
Name: count, dtype: int64
Accuracy: 0.9442
Precision: 0.9447
Recall: 0.9442
F1 Score: 0.9443


In [ ]:
import pandas as pd

# Combine results into a DataFrame
results_df = pd.DataFrame({
    "Model": ["Zero-Shot", "Fine-Tuned"],
    "Accuracy": [zero_shot_evaluation_results["Accuracy"], finetuned_evaluation_results["Accuracy"]],
    "Precision": [zero_shot_evaluation_results["Precision"], finetuned_evaluation_results["Precision"]],
    "Recall": [zero_shot_evaluation_results["Recall"], finetuned_evaluation_results["Recall"]],
    "F1 Score": [zero_shot_evaluation_results["F1 Score"], finetuned_evaluation_results["F1 Score"]],
})

# Save the results to a CSV file
results_file = "final_sst_results.csv"
results_df.to_csv(results_file, index=False)

print(f"Results saved to {results_file}")


Results saved to final_sst_results.csv


## HF hub upload

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load the base model (e.g., "meta-llama/Llama-3.2-1B")
base_model_name = "meta-llama/Llama-3.2-1B"

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Define the output directory for the fine-tuned model
output_dir = "/content/content/llama-3.2-fine-tuned-model"

# Load the PEFT model from the saved directory, passing the base model object
peft_model = PeftModel.from_pretrained(base_model, output_dir)  # Use base_model object here

# Load the tokenizer from the saved directory
tokenizer = AutoTokenizer.from_pretrained(output_dir)


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import notebook_login
from transformers import AutoTokenizer
from peft import PeftModel

# Define the repository name (replace with your actual repo name)
repo_name = "hmhm11901/llama-3.2-sst-finetuned"  # Replace with your desired Hugging Face repo name

# Push the fine-tuned PEFT model to the Hub
peft_model.push_to_hub(repo_name, commit_message="Fine-tuned Llama 3.2 on SST for sentiment analysis.")

# Push the tokenizer to the Hub
tokenizer.push_to_hub(repo_name, commit_message="Tokenizer for fine-tuned Llama 3.2.")


adapter_model.safetensors:   0%|          | 0.00/180M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hmhm11901/llama-3.2-sst-finetuned/commit/e84e00ec4fae66a0a8dbcc923a90c0ec3996ff9d', commit_message='Tokenizer for fine-tuned Llama 3.2.', commit_description='', oid='e84e00ec4fae66a0a8dbcc923a90c0ec3996ff9d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hmhm11901/llama-3.2-sst-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='hmhm11901/llama-3.2-sst-finetuned'), pr_revision=None, pr_num=None)